# Analyze pollen grains using visualize.ClickCount
In this notebook we analyze pollen grain images. A portion of the grains can be detected automatically using a color threshold and filtering the objects according to eccentricity. Then visualize.ClickCount can be used to adjust the detections and manually select the germinated grains.


## Import libraries and define local functions

In [ ]:
%matplotlib widget

from matplotlib import pyplot as plt
from plantcv import plantcv as pcv

In [ ]:
# Input/output options
class options:
    def __init__(self):
        self.image = "./data/example.jpg"  # required
        self.debug = None  # None, "print" (save to file), or "plot" (display in notebook)
        self.writeimg= False
        self.result = "example"  # required
        self.outdir = "./results" # Store the output to the current directory 

# Get options
args = options()

In [ ]:
# Set debug to the global parameter 
pcv.params.debug = args.debug
# Change display settings
pcv.params.dpi = 120
pcv.params.text_size = 10

## Load image and threshold 
The channel used and the threshold value can be adjusted for a better result. The resulting binary image from this step must contain all the grains. The tubes will be removed in the next step

In [ ]:
img, _, fname = pcv.readimage(filename=args.image, mode="native")
h,w,_ = img.shape

pcv.plot_image(img)
plt.title('RGB')

img_B = img[:,:,0]
#pcv.plot_image(img_B)
#plt.title('B channel')

# threshold for the blue channel to adjust for the image
th_B = 150
img_B_thresh = img_B < th_B
#pcv.plot_image(img_B_thresh)
#plt.title('B threshold')

# post-processing for cleaning
img_B_post = pcv.fill_holes(img_B_thresh)
img_B_post = pcv.fill(img_B_post, 210)
pcv.plot_image(img_B_post)
plt.title('Binary image containing all the grains')

## Discard objects that are not circular
Adjust the eccentricity threshold. A value closer to 0 is more strict about the circular form. 
This step is supposed to eliminate the tubes and automatically detect most of the grains. Some grains will be eliminated as well and will have to be selected manually in the next step.

In [ ]:
discs, coor = pcv.detect_discs(img_B_post, ecc_thresh=0.65)
pcv.plot_image(discs)
plt.title('Binary mask after eliminating the tails')

## Initialize interactive tool and import pre-detected grains
Click to select missing grains and to fix bad detections. Adjust figsize for zoom,

In [ ]:
# initialization
counter = pcv.visualize.ClickCount(img, figsize=(8, 6))

# import coordinates (if available)
counter.import_coords(coor, label="total")

# view "total" class
counter.view(label="total", color="r", view_all=False)

## View number of selected points

In [ ]:
print(f"There are {counter.count['total']} selected objects")

## Run interactive tool to select grains with a different class

In [ ]:
counter.view(label="germinated", color="b", view_all=True)

## View number of selected points

In [ ]:
print(f"There are {counter.count['germinated']} selected objects")